<a href="https://colab.research.google.com/github/ryotaro1218/project1_2/blob/main/test_2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd
import numpy as np


# Load data
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
data = pwt1001.loc[pwt1001['country'].isin(["Canada",  "France", "Germany", "Italy", "Japan", "United Kingdom", "United States"])][['year', 'countrycode', 'rgdpna', 'rkna', 'emp', 'labsh', 'rtfpna', 'pop', 'avh', "country"]]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

# Calculate additional columns
data['y_pc'] = np.log(data['rgdpna'] / data['emp'])  # GDP per worker
data['k_pc'] = np.log(data['rkna'] / data['emp'])  # Capital per worker
data['lna'] = np.log(data['rtfpna'])
data['alpha'] = 1 - data['labsh']

# Order by year
data = data.sort_values('year')

# Group by isocode
grouped_data = data.groupby('country')

# Calculate growth rates and Solow residual
data['g'] = (grouped_data['y_pc'].diff() * 100)  # Growth rate of GDP per capita
data['a'] = (grouped_data['lna'].diff() * 100)  # Growth rate of A
data['k'] = (grouped_data['k_pc'].diff() * 100)  # Growth rate of k per capita
data['ak'] = data['k'] * data['alpha']
data['Solow'] = data['g'] - data['a'] - data['ak']
data['ag'] = data['a'] / data['g']
data['akg'] = data['ak'] / data['g']


# Remove missing values
data = data.dropna()

# Calculate summary statistics
summary = data.groupby('country').agg({'g': 'mean', 'alpha': "mean", 'a': "mean", 'ak': "mean", 'Solow': "mean", 'ag': "mean", 'akg': "mean"})

# Calculate additional summary statistics
summary['Growth Rate'] = summary['g']
summary['TFP Growth'] = summary['a']
summary['Capital Deepening'] = summary['ak']
summary['Solow'] = summary['Solow']
summary['A Contribution'] = contributions['ag']
summary['AK Contribution'] = contributions['akg']


summary.loc['Average'] = np.nan
summary.loc['Average', :] = summary.mean()

rounded_summary = summary.round(2)

# Print output
print(rounded_summary)

                   g  alpha     a    ak  Solow  Growth Rate  TFP Growth  \
country                                                                   
Canada          0.89   0.35  0.15  0.63   0.10         0.89        0.15   
France          0.83   0.38  0.09  0.57   0.17         0.83        0.09   
Germany         0.66   0.37  0.36  0.44  -0.14         0.66        0.36   
Italy          -0.04   0.48 -0.77  0.53   0.20        -0.04       -0.77   
Japan           0.71   0.43  0.14  0.63  -0.06         0.71        0.14   
United Kingdom  1.02   0.41  0.23  0.57   0.21         1.02        0.23   
United States   1.54   0.39  0.68  0.78   0.08         1.54        0.68   
Average         0.80   0.40  0.13  0.59   0.08         0.80        0.13   

                Capital Deepening  A Contribution  AK Contribution  
country                                                             
Canada                       0.63            2.13            -1.49  
France                       0.57         